In [ ]:
from concurrent.futures import ThreadPoolExecutor
import yaml

from tqdm import tqdm
import pandas as pd

from openai import OpenAI

import mistake_generator.regex_generator as rg

columns = yaml.safe_load(open("confs.yaml"))["training_set_schema"]

client = OpenAI()

In [36]:
N = 10000
MAX_WORKERS = 6

PROMPT_TEMPLATE = """
A brazilian user trying to search for the company you work for. 
Please generate possible user seach inputs given the correct output of the search (razao social or nome fantasia).
Consider the user input may have typos, missing words, or be incomplete. 
From the user perspective nome fantasia should be more important than razao social. Words like "de", "S.A.", "LTDA" should be ignored.
Also words like "administradora", "geral" should have less importance than "estacionamento" or "restaurante" which is the main business of the company.
Also focus on well known names of the company such as "Drogasil", "Itaú", "Bradesco", "Carrefour", "Pão de Açúcar", "Casas Bahia", "Magazine Luiza" or "MagaLu".

Example of input:
    Output:
        Razão Social: PB ADMINISTRADORA DE ESTACIONAMENTOS LTDA
        Nome fantasia: ROYAL PALM CONTEMPORANEO
    Possible user search inputs: 
            - ROYAL PALM
            - PB ESTACIONAMENTO
            - BP ESTACIONAMENTO
            - ROIAL PALME
            - PALM ROYAL

Fill in the <user search input list> with the possible user search inputs.
    Output:
        Razão Social: {razao_social}
        Nome fantasia: {nome_fantasia}
    Possible user search inputs: 
        <user search input list>
"""

def parse_response(response):
    try:
        lines = response.split("Possible user search inputs:")[1].split("\n")
    except:
        lines = response.split("\n")
    lines = [line.replace("- ","").strip() for line in lines]
    lines = [line for line in lines if line]
    return lines


def get_model_output(
    row: pd.Series,
    prompt_template: str, 
    temperature: float = 0.0, 
    model: str = "gpt-4o-mini"
) -> str:
    
    content = prompt_template.format(razao_social=row['razaosocial'], nome_fantasia=row['nome_fantasia'])

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": content
            }
        ],
        temperature=temperature,
        
    )
    return [completion.choices[0].message.content, row['razaosocial'], row['nome_fantasia']]

def process_row(row):
    return get_model_output(row, PROMPT_TEMPLATE)

In [31]:
df = pd.read_parquet("dados/dataset.parquet")

In [34]:
x = (N/10 * 22) / 3600
print(f"Estimated time to complete: {x:.2f} hours")


Estimated time to complete: 6.11 hours


In [38]:
sample_chance = df.razaosocial.value_counts(normalize=True)

chances = df.merge(sample_chance, left_on='razaosocial', right_index=True)\
    .rename(columns={'razaosocial_y': 'sample_chance'})\
    .sort_values('sample_chance', ascending=False)\
    .drop(columns=['razaosocial_x'])[['sample_chance', 'razaosocial', 'nome_fantasia']]\
    .groupby(['razaosocial', 'nome_fantasia']).sum().sort_values('sample_chance', ascending=False)

# sample without replacement based on the sample_chance
inputs = chances.sample(n=N, replace=False, weights='sample_chance').reset_index().sort_values('sample_chance', ascending=False)

In [40]:
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    results = list(tqdm(executor.map(process_row, [row for _, row in inputs.iterrows()]), total=len(inputs)))

100%|██████████| 10000/10000 [3:14:55<00:00,  1.17s/it]    


In [ ]:
final = pd.DataFrame(results, columns=['response', 'razaosocial', 'nome_fantasia'])
final['parsed_response'] = final.response.apply(parse_response)
final.to_parquet("dados/outputs.parquet")

In [ ]:
full_dataset = final.merge(df, on=['razaosocial', 'nome_fantasia'])
full_dataset = full_dataset.drop(columns="response").explode("parsed_response")
full_dataset.to_parquet("dados/full_dataset.parquet")

In [88]:
full_dataset.head(20)

,razaosocial,nome_fantasia,parsed_response,cnpj,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,uf,capital_social,porte,cnae_fiscal,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,CRUZADA EVANGELICA,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,IGREJA QUADRANGULAR,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,EVANGELHO QUADRANGULAR,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,CRUZADA NACIONAL,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,IGREJA DO EVANGELHO,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,CRUZADA DE EVANGELIZACAO,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,IGREJA CRUZADA,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,EVANGELIZACAO QUADRANGULAR,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,IGREJA NACIONAL,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente
0,IGREJA DO EVANGELHO QUADRANGULAR,CRUZADA NACIONAL DE EVANGELIZACAO,QUADRANGULAR EVANGELHO,62955505457450,filial,20190611,None,20190611,SC,0.0,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente


In [ ]:
df = pd.read_parquet("dados/full_dataset.parquet")
df.reset_index(inplace=True, drop=True)
df['user_input'] = df.parsed_response.apply(lambda x: [x, rg.introduce_typos(x)])
df = df.explode('user_input').drop_duplicates().reset_index(drop=True)
df.to_parquet("dados/full_dataset_with_typos.parquet")

In [ ]:
df = pd.read_parquet("dados/full_dataset_with_typos.parquet")
df.reset_index(inplace=True, drop=True)
df.drop(columns=['parsed_response']).reset_index(inplace=True, drop=True)
df = df[columns]
train = df.sample(frac=0.8, random_state=0)
test = df.drop(train.index)
train.to_parquet("dados/train.parquet")
test.to_parquet("dados/test.parquet")